In [1]:
import numpy as np
from switchingBanditEnv_aistats import SwitchingBanditEnvAistats

In [2]:
num_states = 5
num_actions = 18
num_obs = 5
delta = 0.1

observation_multiplier = 20
# transition_multiplier = 15
transition_multiplier = 20
num_experiments = 5

In [4]:
switching = SwitchingBanditEnvAistats(num_states=num_states,
                               num_actions=num_actions,
                               num_obs=num_obs,
                               transition_multiplier=transition_multiplier,
                               observation_multiplier=observation_multiplier)
real_transition_matrix = switching.transition_matrix

Sigma min is 0.22172564526595007
Sigma min is 0.008571427227870377
Sigma min is 0.00524679809448345
Sigma min is 0.005771449220019451
Sigma min is 0.00648543247757683
Sigma min is 0.20117900427507796
Sigma min is 0.0016836340246243811
Sigma min is 0.00804692106009894
Sigma min is 0.020544691169926695
Sigma min is 0.018327967082612386
Sigma min is 0.005361862885172478
Sigma min is 0.014402990586188416
Sigma min is 0.0071839711873053465
Sigma min is 0.08448991404082226
Sigma min is 0.014518920989939229
Sigma min is 0.005318938750673532
Sigma min is 0.007574658157369556
Sigma min is 0.0029523220866050793
(5, 5)
Kronecker sigma min is 0.04916226176860196
Product sigma min is 0.049162261768601924
(5, 5)
Kronecker sigma min is 0.0019005052329497008
Product sigma min is 0.0019005052329496928
(5, 5)
Kronecker sigma min is 0.0011633496930795381
Product sigma min is 0.0011633496930795002
(5, 5)
Kronecker sigma min is 0.0012796783024284699
Product sigma min is 0.001279678302428477
(5, 5)
Kronecke

In [8]:
random_matrix = np.random.random(size=(num_states, num_states)) * 0.1
perturbed_transition_matrix = real_transition_matrix + random_matrix
perturbed_transition_matrix = perturbed_transition_matrix / perturbed_transition_matrix.sum(axis=1)[:, None]
print(f"Real transition matrix is {real_transition_matrix}")
print(f"Perturbed transition matrix is {perturbed_transition_matrix}")

transition_distance = np.absolute(real_transition_matrix.reshape(-1) -
                              perturbed_transition_matrix.reshape(-1))
print(f"Distance vector is {abs(np.sum(transition_distance))}")

Real transition matrix is [[0.54210526 0.01578947 0.12631579 0.18947368 0.12631579]
 [0.21634615 0.59615385 0.04326923 0.01442308 0.12980769]
 [0.04663212 0.17098446 0.70466321 0.06217617 0.01554404]
 [0.17030568 0.11790393 0.03930131 0.6069869  0.06550218]
 [0.1640625  0.1640625  0.10546875 0.12890625 0.4375    ]]
Perturbed transition matrix is [[0.447583   0.02457175 0.17705212 0.18838727 0.16240586]
 [0.20179412 0.52296069 0.08595904 0.02896905 0.16031709]
 [0.04152947 0.18260469 0.59280221 0.11507306 0.06799057]
 [0.14236706 0.13747211 0.07743499 0.5421664  0.10055944]
 [0.1776935  0.16685033 0.15763614 0.12023754 0.37758249]]
Distance vector is 0.9233257122986802


In [25]:
belief = np.ones(shape=num_states) * 1/num_states
cumulated_difference = 0
all_prob = 0
distance_vec = np.zeros(shape=num_states)
for first_state in range(num_states):
    for second_state in range(num_states):
        real_prob = belief[first_state] * real_transition_matrix[first_state, second_state]
        all_prob += real_prob
        perturbed_prob = belief[first_state] * perturbed_transition_matrix[first_state, second_state]
        distance = real_prob - perturbed_prob
        distance_vec[second_state] += distance

print(distance_vec)
belief_distance = np.absolute(distance_vec)
print(f"Sum of distances is {np.sum(belief_distance)}")

[ 0.02569691  0.00608693 -0.01437324  0.00142655 -0.01883715]
Sum of distances is 0.06642078137007809


In [21]:
belief = np.ones(shape=num_states) * 1/num_states
cumulated_difference = 0
all_prob = 0
final_belief = np.zeros(shape=num_states)
for first_state in range(num_states):
    for second_state in range(num_states):
        for third_state in range(num_states):
            real_prob = belief[first_state] * real_transition_matrix[first_state, second_state] * real_transition_matrix[second_state, third_state]
            final_belief[third_state] += real_prob

print(final_belief)

[0.23866846 0.21445968 0.20583093 0.20053077 0.14051017]


In [22]:
after_first_belief = np.dot(belief, real_transition_matrix)
after_second_belief = np.dot(after_first_belief, real_transition_matrix)
print(after_second_belief)

[0.23866846 0.21445968 0.20583093 0.20053077 0.14051017]


In [19]:
real_belief = np.dot(belief, real_transition_matrix)
print(real_belief)
perturbed_belief = np.dot(belief, perturbed_transition_matrix)
print(perturbed_belief)

belief_distance = np.absolute(real_belief.reshape(-1) -
                              perturbed_belief.reshape(-1))
print(f"Distance vector is {np.sum(belief_distance)}")

belief_distance = real_belief.reshape(-1) - perturbed_belief.reshape(-1)
print(f"Distance vector is {belief_distance}")


[0.22789034 0.21297884 0.20380366 0.20039322 0.15493394]
[0.20219343 0.20689191 0.2181769  0.19896666 0.17377109]
Distance vector is 0.06642078137007806


In [30]:
real = real_transition_matrix
perturbed_transition_matrix = switching.generate_transition_matrix_v2(transition_multiplier=20)
perturbed = perturbed_transition_matrix
distance = np.absolute(real_transition_matrix.reshape(-1) -
                          perturbed_transition_matrix.reshape(-1))
print(f"Original distance is {np.sum(distance)}")

for i in range(20):
    real = np.dot(real, real_transition_matrix)
    perturbed = np.dot(perturbed, perturbed_transition_matrix)
    distance = np.absolute(real.reshape(-1) -
                              perturbed.reshape(-1))
    print(f"Distance vector is {np.sum(distance)}")

Original distance is 1.9881915103578964
Distance vector is 2.2084628286957586
Distance vector is 2.000540563312738
Distance vector is 1.7331799133806454
Distance vector is 1.546154574359751
Distance vector is 1.4615472078743055
Distance vector is 1.4324649367456694
Distance vector is 1.4441218373425508
Distance vector is 1.4631003663278965
Distance vector is 1.476927065942491
Distance vector is 1.4870632075173718
Distance vector is 1.4945247869754958
Distance vector is 1.5000323654212233
Distance vector is 1.5041045438423328
Distance vector is 1.5071184993776963
Distance vector is 1.509350530211577
Distance vector is 1.511003999725247
Distance vector is 1.5122290419926054
Distance vector is 1.5131366987081099
Distance vector is 1.5138091859015383
Distance vector is 1.5143074118460875
